###Денежно нейтральный спред
    Пример 1
    Денежно нейтральный спред для фьючерсов на евро и доллар.

    На входе:
        Источники данных:
            Фьючерс на доллар Si       [S/R] = [Доллар США/Рубль]
            Фьючерс на евро   Eu       [Е/R] = [Евро/Рубль]
            Фьючерс на евро/доллар Ed  [Е/S] = [Евро/Доллар США]
            
            Текущие ближайшие фьючерсы:
            Si-6.15(SiM5)
            Eu-6.15(EuM5)
            ED-6.15(EDM5)

        Официальные коды и наименования:
        
http://legacy.moex.com/marketdata/quotes            

        S  = Nb*PB - Nq*PQ -> 0
        PS = Nb*PB + Nq*PQ        
        
        S - cпред как разность
        PS - суммарная стоимость базового портфеля и котируемого портфеля 
        
    Задано:
        Обозначения:
            Eu - стоимость Фьючерс на евро
            Si - стоимость Фьючерс на доллар Si
            Ed - стоимость Фьючерс на евро/доллар 

        Kbq = P_B/P_Q
        (где P_B P_Q - средние на некотором интервале цены базового и котируемого портфеля)
        В нашем случае можем определять Kbq как среднее от Ed:
        
        Kbq = xMA(Ed,n) 
            где xMA - некоторая скользящая средняя (MA) с периодом n
        n_Kbq - периодом для скользящая средняя Kbq

        Nb - число  единиц базового портфеля
        Nb = 10                           
        Nq - число единиц котируемого портфеля        
        Nq = Nb*P_B/P_Q = Nb*Kbq   
        
        PB - стоимости базового портфеля 
        PB = Eu
        
        PQ - стоимости котируемого портфеля
        PQ = Si        

        Денежно нейтральный спред для фьючерсов на евро и доллар:
        Xb = 1/2 - доля котируемого портфеля
        Xq = 1/2 - доля базового портфеля 

        St = 1*PBt - Kbq*PQt = [Е/R] - [Е/S]*[S/R] = [Е/R]- [Е/R]

        Оклонение спреда от среднего в момпнт времени t:
        DSt = St - xMA(St,n)
            где xMA - некоторая скользящая средняя (MA) с периодом n от спреда S        
        n_St - периодом для скользящая средняя по St

        lev           - количество уровней в каждую сторону от нулевого
        L = lev*2 + 1 - всего уровней котирования с учетом нулевого
        dL -          - шаг уровня
        vL            - список значений для уровней L

-----------------
                
                Пример L = 7 dL = 2
                --------------------------
                |Уровнь   | Значение     |
                --------------------------
                Уровень+3    6            
                Уровень+2    4            
                Уровень+1    2            
                Уровень 0    0            
                Уровень-1   -2            
                Уровень-2   -4            
                Уровень-3   -6            

                Наименование уровня отражает направление отлонения спреда от средней.
-----------------        
    Операции:                    
            Продажа Сперда -S
            -S = -Nb*PB + Nq*PQ
            т.е. продажа Nb*PB и покупка Nq*PQ
            
            Покупка Сперда +S            
            +S = +Nb*PB - Nq*PQ    
            т.е. покупка Nb*PB и продажа Nq*PQ
            
            (Первая операция может быть лимитная (заявкой в стакан), при ее выполнении - вторая по рынку).
-----------------        

    Алгоритм:
            Старт стратегии:
                Lc = 0 - уровень текущий
            Проверка достижения значением отклонения нового уровня
                LnU и LnD - новый уровень
                LnU = max № где DSt >= vL
                LnD = min № где DSt <= vL 

            Проверка условия открытия(закрытия) позиции
                Если DSt >= vL и LnU > Lc                   
                    Продажа Сперда -S {
                        Определение Nq = Nb*Kbq (целое)
                        запомнить Nb (задан) Nq (может меняться)
                    }
                    или Закрытие предыдущей позиции {
                        извлечение Nb Nq                    
                    }
                    
                    Lc = LnU                     
                Если DSt <= vL и LnD < Lc                                        
                    Покупка Сперда +S {
                        Определение Nq = Nb*Kbq (целое)                    
                        запомнить Nb (задан) Nq (может меняться)
                    }
                    или Закрытие предыдущей позиции {
                        извлечение Nb Nq                                        
                    }
                    Lc = LnU                     
            Стоп стратегии:
                Закрытие текущих позиций            
            
-----------------        
    Пример : Алгоритм торговли нейтральным спредом
 	---------------------------------------------------------------------------------- 	
    |         |              | Действие если       |             | Действие если 
    |         | Значение     | Предыдущий уровень  | Значение    | Предыдущий уровень   
    |Уровнь   | ниже         | выше                | выше        | ниже
    ----------------------------------------------------------------------------------
    Уровень+3    -             -                      6             Продажа S
    Уровень+2    4             Покупка S Закрытие     4             Продажа S
    Уровень+1    2             Покупка S Закрытие     2             Продажа S
    Уровень 0    0             Покупка S Закрытие     0             Продажа S Закрытие
    Уровень-1   -2             Покупка S             -2             Продажа S Закрытие
    Уровень-2   -4             Покупка S             -4             Продажа S Закрытие
    Уровень-3   -6             Покупка S             -              -
-----------------        


![Многоуровневый Маркет мейкинг](DeviationLKOHSIBN2.jpg "Многоуровневый Маркет мейкинг")

    Экспорт котировок для фьючерсов на евро и доллар:
    
http://www.finam.ru/analysis/profile636AE00007/?market=5

http://www.finam.ru/analysis/profile6145500007/?market=14

http://www.finam.ru/analysis/profile5E41200007/?market=14

    
http://www.finam.ru/analysis/profile636AE/?market=5

http://www.finam.ru/analysis/profile61455/?market=14

http://www.finam.ru/analysis/profile5E412/?market=14

http://www.micex.ru/markets/futures/section/instruments/currency_futures/1263

http://legacy.moex.com/marketdata/quotes